In [1]:
import pandas as pd
import numpy as np

### Concatenate and transform pitches dataframes

In [48]:
games_2013 = pd.read_csv("games_2013.csv")
games_2013.head()

,Unnamed: 0,pitch,balls,strikes,count,batter,pitcher,umpire,home_pitcher,inning,run_diff
0,0,C,0,0,0-0,Ian Kinsler,Bud Norris,Sam Holbrook,1,1,0
1,1,X,0,1,0-1,Ian Kinsler,Bud Norris,Sam Holbrook,1,1,0
2,2,B,0,0,0-0,Elvis Andrus,Bud Norris,Sam Holbrook,1,1,0
3,3,S,1,0,1-0,Elvis Andrus,Bud Norris,Sam Holbrook,1,1,0
4,4,C,1,1,1-1,Elvis Andrus,Bud Norris,Sam Holbrook,1,1,0


In [49]:
games_2014 = pd.read_csv("games_2014.csv")
games_2014.head()

,Unnamed: 0,pitch,balls,strikes,count,batter,pitcher,umpire,home_pitcher,inning,run_diff
0,0,C,0,0,0-0,Yasiel Puig,Wade Miley,Tim Welke,1,1,0
1,1,F,0,1,0-1,Yasiel Puig,Wade Miley,Tim Welke,1,1,0
2,2,S,0,2,0-2,Yasiel Puig,Wade Miley,Tim Welke,1,1,0
3,3,B,0,0,0-0,Justin Turner,Wade Miley,Tim Welke,1,1,0
4,4,B,1,0,1-0,Justin Turner,Wade Miley,Tim Welke,1,1,0


In [50]:
games_2015 = pd.read_csv("games_2015.csv")

games_2015.head()

,Unnamed: 0,pitch,balls,strikes,count,batter,pitcher,umpire,home_pitcher,inning,run_diff
0,0,C,0,0,0-0,Matt Carpenter,Jon Lester,Mike Winters,1,1,0
1,1,S,0,1,0-1,Matt Carpenter,Jon Lester,Mike Winters,1,1,0
2,2,F,0,2,0-2,Matt Carpenter,Jon Lester,Mike Winters,1,1,0
3,3,B,0,2,0-2,Matt Carpenter,Jon Lester,Mike Winters,1,1,0
4,4,B,1,2,1-2,Matt Carpenter,Jon Lester,Mike Winters,1,1,0


In [51]:
# Combine games 2013 - 2015
# Drop unnecessary columns
pitches_df = pd.concat([games_2013, games_2014, games_2015])\
                .drop(columns=['Unnamed: 0', 'balls', 'strikes', 'batter'])

pitches_df.head()

,pitch,count,pitcher,umpire,home_pitcher,inning,run_diff
0,C,0-0,Bud Norris,Sam Holbrook,1,1,0
1,X,0-1,Bud Norris,Sam Holbrook,1,1,0
2,B,0-0,Bud Norris,Sam Holbrook,1,1,0
3,S,1-0,Bud Norris,Sam Holbrook,1,1,0
4,C,1-1,Bud Norris,Sam Holbrook,1,1,0


In [52]:
# Drop all rows where pitch is not a called pitch
pitches_df = pitches_df[(pitches_df.pitch == 'C') | (pitches_df.pitch == 'B')]
pitches_df.head()

,pitch,count,pitcher,umpire,home_pitcher,inning,run_diff
0,C,0-0,Bud Norris,Sam Holbrook,1,1,0
2,B,0-0,Bud Norris,Sam Holbrook,1,1,0
4,C,1-1,Bud Norris,Sam Holbrook,1,1,0
5,B,1-2,Bud Norris,Sam Holbrook,1,1,0
7,B,0-0,Bud Norris,Sam Holbrook,1,1,0


In [53]:
# Convert pitch column to True/False - whether pitch was a strike
# C = 'C'alled strike
# B = 'B'all (not strike)
pitches_df['strike_given_called'] = (pitches_df.pitch == 'C')
pitches_df.head()

,pitch,count,pitcher,umpire,home_pitcher,inning,run_diff,strike_given_called
0,C,0-0,Bud Norris,Sam Holbrook,1,1,0,True
2,B,0-0,Bud Norris,Sam Holbrook,1,1,0,False
4,C,1-1,Bud Norris,Sam Holbrook,1,1,0,True
5,B,1-2,Bud Norris,Sam Holbrook,1,1,0,False
7,B,0-0,Bud Norris,Sam Holbrook,1,1,0,False


In [54]:
# Drop pitch column
pitches_df = pitches_df.drop(columns=['pitch'])
pitches_df.head()

,count,pitcher,umpire,home_pitcher,inning,run_diff,strike_given_called
0,0-0,Bud Norris,Sam Holbrook,1,1,0,True
2,0-0,Bud Norris,Sam Holbrook,1,1,0,False
4,1-1,Bud Norris,Sam Holbrook,1,1,0,True
5,1-2,Bud Norris,Sam Holbrook,1,1,0,False
7,0-0,Bud Norris,Sam Holbrook,1,1,0,False


### Merge with pitcher data to get race of pitcher

In [55]:
# Get data from MLB census 2014
mlb_census_df = pd.read_csv("mlb_census.csv")
mlb_census_df.head()

,#,NAME,POS,POS (BROAD),BAT,THW,AGE,Height,WT,BIRTH CITY,BIRTH STATE,BIRTH COUNTRY,RACE,Experience,SCHOOL,SALARY,TEAM
0,13,Alex Avila,C,Catcher,L,R,27,71,210,"Hialeah, FL",FL,United States,Hispanic,5.0,Alabama,"$4,150,000",Detroit Tigers
1,30,David Robertson,RP,Pitcher,R,R,29,71,195,"Birmingham, AL",AL,United States,White,6.0,Alabama,"$5,215,000",New York Yankees
2,29,Tommy Hunter,RP,Pitcher,R,R,27,75,248,"Indianapolis, IN",IN,United States,White,6.0,Alabama,"$3,000,000",Orioles
3,20,Anthony Recker,C,Catcher,R,R,30,74,240,"Allentown, PA",PA,United States,White,3.0,Alvernia College,"$505,340",New York Mets
4,4,Nick Hundley,C,Catcher,R,R,30,73,196,"Corvallis, OR",OR,United States,White,6.0,Arizona,"$4,000,000",San Diego Padres


In [56]:
mlb_census_df = mlb_census_df[['NAME', 'RACE']].rename(columns={'NAME': 'pitcher_name', 'RACE': 'pitcher_race'})
mlb_census_df.head()

,pitcher_name,pitcher_race
0,Alex Avila,Hispanic
1,David Robertson,White
2,Tommy Hunter,White
3,Anthony Recker,White
4,Nick Hundley,White


In [57]:
mlb_census_df.pitcher_race = mlb_census_df.pitcher_race.apply(lambda x : x.lower())

In [58]:
mlb_census_df.head()

,pitcher_name,pitcher_race
0,Alex Avila,hispanic
1,David Robertson,white
2,Tommy Hunter,white
3,Anthony Recker,white
4,Nick Hundley,white


In [59]:
# Get researched data on other pitchers' races
missing_pitcher_df = pd.read_csv("pitcher_race.csv")
missing_pitcher_df.head()

,pitcher_name,pitcher_race
0,Aaron Sanchez,hispanic
1,Alex Burnett,hispanic
2,Alex Claudio,hispanic
3,Alex Sanabia,hispanic
4,Andy Pettitte,white


In [60]:
missing_pitcher_df.pitcher_race.value_counts()

white        455
black         91
hispanic      87
asian         14
hispanic       1
Name: pitcher_race, dtype: int64

In [61]:
# Fix an extra space in an entry
missing_pitcher_df.pitcher_race = missing_pitcher_df.pitcher_race.apply(lambda x : x.replace("hispanic ", "hispanic"))

In [62]:
all_pitchers = pd.concat([missing_pitcher_df, mlb_census_df])
all_pitchers.head()

,pitcher_name,pitcher_race
0,Aaron Sanchez,hispanic
1,Alex Burnett,hispanic
2,Alex Claudio,hispanic
3,Alex Sanabia,hispanic
4,Andy Pettitte,white


In [90]:
all_pitchers.pitcher_race.value_counts()

white       905
hispanic    302
black       162
asian        29
Name: pitcher_race, dtype: int64

In [63]:
# Perform left join on pitcher_name
# First, we must prepare
pitches_df = pitches_df.rename(columns={'pitcher': 'pitcher_name', 'umpire': 'umpire_name'})
pitches_df.head()

,count,pitcher_name,umpire_name,home_pitcher,inning,run_diff,strike_given_called
0,0-0,Bud Norris,Sam Holbrook,1,1,0,True
2,0-0,Bud Norris,Sam Holbrook,1,1,0,False
4,1-1,Bud Norris,Sam Holbrook,1,1,0,True
5,1-2,Bud Norris,Sam Holbrook,1,1,0,False
7,0-0,Bud Norris,Sam Holbrook,1,1,0,False


In [64]:
# We have to fix a character encoding problem
set(pitches_df.pitcher_name)

{'Chris\xa0Resop',
 'Dellin\xa0Betances',
 'John\xa0Baker',
 'Hansel\xa0Robles',
 'Keith\xa0Butler',
 'Joaquin\xa0Benoit',
 'Clay\xa0Rapada',
 'Danny\xa0Duffy',
 'Jon\xa0Garland',
 'Justin\xa0Germano',
 'Jeff\xa0Francis',
 'Collin\xa0Balester',
 'Jim\xa0Henderson',
 'Rafael\xa0Montero',
 'Eric\xa0Jokisch',
 'Rick\xa0Porcello',
 'Frankie\xa0Montas',
 'Daniel\xa0Corcino',
 'Erik\xa0Goeddel',
 'Cody\xa0Allen',
 'Blake\xa0Parker',
 'Adam\xa0Morgan',
 'Diego\xa0Moreno',
 'Casey\xa0Sadler',
 'Leury\xa0Garcia',
 'Casey\xa0Coleman',
 'Donn\xa0Roach',
 'Lester\xa0Oliveros',
 'Garrett\xa0Jones',
 'Leonel\xa0Campos',
 'Javy\xa0Guerra',
 'Steven\xa0Matz',
 'Colby\xa0Lewis',
 'Jeremy\xa0Affeldt',
 'Josh\xa0Wall',
 'Neal\xa0Cotts',
 'Ethan\xa0Martin',
 "Sean\xa0O'Sullivan",
 'Caleb\xa0Thielbar',
 'Chris\xa0Gimenez',
 'Wily\xa0Peralta',
 'Adam\xa0Wilk',
 'John\xa0Axford',
 'Xavier\xa0Cedeno',
 'Jerry\xa0Blevins',
 'Tyler\xa0Skaggs',
 'Brad\xa0Mills',
 'David\xa0Rollins',
 'James\xa0McDonald',
 'Casey

In [66]:
pitches_df.pitcher_name = pitches_df.pitcher_name.apply(lambda x: x.replace('\xa0', ' '))

In [78]:
set(pitches_df.pitcher_name) - set(all_pitchers.pitcher_name)

{"Eric O'Flaherty", "Ryan O'Rourke", "Sean O'Sullivan"}

In [81]:
set(pitches_df.pitcher_name) - set(all_pitchers.pitcher_name.apply(lambda x: x.replace("OFlaherty", "O'Flaherty"))\
                                  .apply(lambda x: x.replace("ORourke", "O'Rourke"))\
                                  .apply(lambda x: x.replace("OSullivan", "O'Sullivan")))

set()

In [83]:
all_pitchers.pitcher_name = all_pitchers.pitcher_name.apply(lambda x: x.replace("OFlaherty", "O'Flaherty"))\
                                  .apply(lambda x: x.replace("ORourke", "O'Rourke"))\
                                  .apply(lambda x: x.replace("OSullivan", "O'Sullivan"))

In [84]:
merge1_df = pd.merge(pitches_df, all_pitchers, how='left', on='pitcher_name')
merge1_df.head()

,count,pitcher_name,umpire_name,home_pitcher,inning,run_diff,strike_given_called,pitcher_race
0,0-0,Bud Norris,Sam Holbrook,1,1,0,True,white
1,0-0,Bud Norris,Sam Holbrook,1,1,0,False,white
2,1-1,Bud Norris,Sam Holbrook,1,1,0,True,white
3,1-2,Bud Norris,Sam Holbrook,1,1,0,False,white
4,0-0,Bud Norris,Sam Holbrook,1,1,0,False,white


In [86]:
any(merge1_df.pitcher_race.isna())

False

### Merge with umpire data to get umpire race

In [87]:
# Load umpire data
umpire_df = pd.read_csv("umpire_race.csv")
umpire_df.head()

,Unnamed: 0,umpire_name,umpire_race
0,0,Jordan Baker,white
1,1,Lance Barksdale,white
2,2,Lance Barrett,white
3,3,Ted Barrett,white
4,4,Scott Barry,white


In [89]:
umpire_df.umpire_race.value_counts()

white       88
black        6
hispanic     6
Name: umpire_race, dtype: int64

In [95]:
umpire_df = umpire_df[['umpire_name', 'umpire_race']]
umpire_df.head()

,umpire_name,umpire_race
0,Jordan Baker,white
1,Lance Barksdale,white
2,Lance Barrett,white
3,Ted Barrett,white
4,Scott Barry,white


In [102]:
set(merge1_df.umpire_name) - set(umpire_df.umpire_name)

{"Brian O'Nora"}

In [104]:
umpire_df[umpire_df.umpire_name == 'Brian ONora']

,umpire_name,umpire_race
59,Brian ONora,white


In [106]:
umpire_df.umpire_name = umpire_df.umpire_name.apply(lambda x: x.replace("Brian ONora", "Brian O'Nora"))

In [107]:
merge2_df = pd.merge(merge1_df, umpire_df, how='left', on='umpire_name')
merge2_df.head()

,count,pitcher_name,umpire_name,home_pitcher,inning,run_diff,strike_given_called,pitcher_race,umpire_race
0,0-0,Bud Norris,Sam Holbrook,1,1,0,True,white,white
1,0-0,Bud Norris,Sam Holbrook,1,1,0,False,white,white
2,1-1,Bud Norris,Sam Holbrook,1,1,0,True,white,white
3,1-2,Bud Norris,Sam Holbrook,1,1,0,False,white,white
4,0-0,Bud Norris,Sam Holbrook,1,1,0,False,white,white


In [108]:
merge2_df[merge2_df.umpire_race.isna()]

,count,pitcher_name,umpire_name,home_pitcher,inning,run_diff,strike_given_called,pitcher_race,umpire_race


In [111]:
# Finally, drop extra columns and convert strike_given_called to 1 or 0 to save space
merge2_df = merge2_df.drop(columns=['pitcher_name', 'umpire_name'])

In [113]:
merge2_df.strike_given_called = merge2_df.strike_given_called.apply(int)

In [114]:
merge2_df.head()

,count,home_pitcher,inning,run_diff,strike_given_called,pitcher_race,umpire_race
0,0-0,1,1,0,1,white,white
1,0-0,1,1,0,0,white,white
2,1-1,1,1,0,1,white,white
3,1-2,1,1,0,0,white,white
4,0-0,1,1,0,0,white,white


In [115]:
# Write out csv
merge2_df.to_csv("cp_merged.csv", index=False)